In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
import pyodbc 
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=(Local);'
                      'Database=LogicUniversity;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('SELECT dbo.Orders.POnumber,DateOrdered,dbo.OrderDetails.ItemCode,dbo.OrderDetails.Quantity FROM dbo.Orders INNER JOIN dbo.OrderDetails on dbo.Orders.POnumber=dbo.OrderDetails.POnumber')

for row in cursor:
    print(row)

(99990001, datetime.datetime(2019, 1, 6, 0, 0), 'C001', 62.0)
(99990001, datetime.datetime(2019, 1, 6, 0, 0), 'E002', 67.0)
(99990001, datetime.datetime(2019, 1, 6, 0, 0), 'E035', 16.0)
(99990001, datetime.datetime(2019, 1, 6, 0, 0), 'F033', 94.0)
(99990001, datetime.datetime(2019, 1, 6, 0, 0), 'P032', 94.0)
(99990002, datetime.datetime(2019, 2, 6, 0, 0), 'C002', 29.0)
(99990002, datetime.datetime(2019, 2, 6, 0, 0), 'E003', 28.0)
(99990002, datetime.datetime(2019, 2, 6, 0, 0), 'E036', 76.0)
(99990002, datetime.datetime(2019, 2, 6, 0, 0), 'F034', 67.0)
(99990002, datetime.datetime(2019, 2, 6, 0, 0), 'P033', 67.0)
(99990003, datetime.datetime(2019, 3, 6, 0, 0), 'C001', 55.0)
(99990003, datetime.datetime(2019, 3, 6, 0, 0), 'C003', 66.0)
(99990003, datetime.datetime(2019, 3, 6, 0, 0), 'E004', 62.0)
(99990003, datetime.datetime(2019, 3, 6, 0, 0), 'F020', 28.0)
(99990003, datetime.datetime(2019, 3, 6, 0, 0), 'F035', 28.0)
(99990003, datetime.datetime(2019, 3, 6, 0, 0), 'P034', 28.0)
(9999000

In [13]:
SQL_Query = pd.read_sql_query(
'''SELECT dbo.Orders.POnumber,DateOrdered,dbo.OrderDetails.ItemCode,dbo.OrderDetails.Quantity FROM dbo.Orders INNER JOIN dbo.OrderDetails on dbo.Orders.POnumber=dbo.OrderDetails.POnumber''', conn)

In [14]:
df = pd.DataFrame(SQL_Query, columns=['POnumber','DateOrdered','ItemCode','Quantity'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 4 columns):
POnumber       322 non-null int64
DateOrdered    322 non-null datetime64[ns]
ItemCode       322 non-null object
Quantity       322 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 10.1+ KB


In [15]:
for x in range(0,df.shape[0]):
    code=df.at[x,'ItemCode']
    init = (ord(code[0])%32)
    df.set_value(x, 'ItemCode', int(str(init)+code[1:]))

C:\Users\A0078\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [16]:
df.head()

,POnumber,DateOrdered,ItemCode,Quantity
0,99990001,2019-01-06,3001,62.0
1,99990001,2019-01-06,5002,67.0
2,99990001,2019-01-06,5035,16.0
3,99990001,2019-01-06,6033,94.0
4,99990001,2019-01-06,16032,94.0


In [17]:
#extract only the month
df.DateOrdered = [item.month for item in df.DateOrdered] 

In [18]:
df.head()

,POnumber,DateOrdered,ItemCode,Quantity
0,99990001,1,3001,62.0
1,99990001,1,5002,67.0
2,99990001,1,5035,16.0
3,99990001,1,6033,94.0
4,99990001,1,16032,94.0


In [19]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

x = df[['ItemCode','DateOrdered']]
y = df['Quantity'] # This is the same as
#y = df.Sales

# default split ratio is 25% for test set
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state = 1)

In [20]:
x_train.head()

,ItemCode,DateOrdered
39,16038,7
271,5033,8
78,5001,3
201,19011,9
147,16012,8


In [21]:
y_train.head()

39     50.0
271    60.0
78     57.0
201    90.0
147    35.0
Name: Quantity, dtype: float64

In [27]:
# import model
#from sklearn.linear_model import LinearRegression

#instantiate
#linReg = LinearRegression()

# fit out linear model to the train set data
#linReg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
# import the model from sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# we choose k = 3 here, due to the small number of data we have
# there isn't a k suitable for all. Generally we want to choose a small odd number.
# large k is generally more costy
knn_model = KNeighborsClassifier(n_neighbors = 3) 

knn_model.fit(x_train, y_train) # train our model to the data set we have. The training set is all the data points

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [23]:
#y_hat = linReg.predict(x_test)
y_hat = knn_model.predict(x_test)


In [24]:
from sklearn.metrics import mean_squared_error, r2_score

# the error is large due to the data in our set are large values, we should use r2-score to check
mean_squared_error(y_test, y_hat) 

2091.3086419753085

In [25]:
r2_score(y_test, y_hat)

-1.262024801849308

In [27]:
import pickle
#pickle.dump(linReg, open('LogicUniversity_LinReg.pkl', 'wb'))
pickle.dump(knn_model, open('LogicUniversity_KNN.pkl', 'wb'))